# Using GPT-3 to decompose a task to relevant virtualhome instructions

## Import Libraries

In [1]:
import sys
import re
sys.path.append('../../simulation')
from unity_simulator.comm_unity import UnityCommunication

## Setup Open-AI and Connect to Unity

In [3]:
# Connect to GPT-3
import openai

# Get the API key from the environment variable
import os
openai.api_key = os.environ["OPENAI_API_KEY"]


In [90]:
# Setting up Unity
comm = UnityCommunication()
comm.reset()
comm.add_character('Chars/Female1')
print('Unity is ready')

# Load the actions from the file
with open('actions.txt', 'r') as f:
    actions = f.read().splitlines()

Unity is ready


## Define Prompt Creators

In [91]:
def get_rooms_and_items(comm):
    # Get environment graph from Unity
    env_graph = comm.environment_graph()

    # Get a dict of rooms:objects from the environment graph
    rooms_items = {}
    present_class = None
    for idx, node in enumerate(env_graph[1]['nodes']):
        if 'ROO_' in node['prefab_name']:
            present_class = node['class_name']
            # Create an empty set for the room
            rooms_items[present_class] = {"idx": node['id'],  "objects": {}}
        else:
            if present_class is None:
                continue
            if rooms_items[present_class] is not None:
                obj_class_name = node['class_name']
                rooms_items[present_class]["objects"][obj_class_name] = node['id']

    rooms = list(rooms_items.keys())
    return rooms, rooms_items

In [92]:
rooms, rooms_items = get_rooms_and_items(comm)

In [106]:
def create_room_prompt(rooms, task):
    prompt = """
    Available rooms: {rooms}
    Task: Watch TV
    Relevant Room: livingroom

    Available rooms: {rooms}
    Task: Grab a plate
    Relevant Room: kitchen

    Available rooms: {rooms}
    Task: Wash clothes
    Relevant Room: bathroom

    Available rooms: {rooms}
    Task: {task}
    Relevant Room:
    """.format(rooms=rooms, task=task)
    return prompt
    

In [131]:
def create_execution_prompt(rooms_items, task, relevant_room):
    objects_in_room = rooms_items[relevant_room]["objects"]
    prompt2 = """
Available actions: {actions}

Available objects: {objects_livingroom}
Relevant Room: livingroom
Task: Watch TV
Execution:
<char0> [walk] <livingroom> (336)
<char0> [walk] <tv> (427)
<char0> [switchon] <tv> (427)

Available objects: {objects_kitchen}
Relevant Room: kitchen
Task: Grab a plate
Execution:
<char0> [walk] <kitchen> (207)
<char0> [find] <plate> (315)
<char0> [grab] <plate> (315)

Available objects: {objects_bathroom}
Relevant Room: bathroom
Task: Wash clothes
Execution:
<char0> [walk] <clothespile> (170)
<char0> [grab] <clothespile> (170)
<char0> [open] <washingmachine> (73)
<char0> [putin] <clothespile> (179) <washingmachine> (73)
<char0> [close] <washingmachine> (73)
<char0> [switchon] <washingmachine> (73)

Available objects: {objects2}
Relevant Room: {relevant_room}
Task: {task}
Execution:
""".format( 
        actions=actions,
        objects_livingroom=rooms_items["livingroom"]["objects"], 
        objects_kitchen=rooms_items["kitchen"]["objects"], 
        objects_bathroom=rooms_items["bathroom"]["objects"],
        objects2=objects_in_room, 
        relevant_room=relevant_room,
        task=task
    )
    return prompt2

## Get the script for a high level task

In [132]:
task = "Microwave salmon"

def do_task(task, rooms_items):
    room_response = openai.Completion.create(
        model="text-davinci-002",
        prompt=create_room_prompt(rooms, task),
        temperature=0.7,
        max_tokens=256,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0
    )
    relevant_room = room_response.choices[0].text.strip()
    print("Relevant room: ", relevant_room)

    response = openai.Completion.create(
        model="text-davinci-002",
        prompt=create_execution_prompt(rooms_items, task, relevant_room),
        temperature=0.7,
        max_tokens=256,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0
    )
    script = response.choices[0].text.strip().split('\n')
    return script

script = do_task(task, rooms_items)

Relevant room:  kitchen


In [133]:
script

['<char0> [walk] <kitchen> (207)',
 '<char0> [find] <salmon> (328)',
 '<char0> [grab] <salmon> (328)',
 '<char0> [open] <microwave> (314)',
 '<char0> [putin] <salmon> (328) <microwave> (314)',
 '<char0> [close] <microwave> (314)',
 '<char0> [switchon] <microwave> (314)']

## Render the script

In [134]:
comm.render_script(script, recording=True, find_solution=True)

(True, {'0': {'message': 'Success'}})